In [68]:
#Vamos importar os módulos que precisamos
from cmath import *
from numpy import linalg
import numpy as np

# CIRCUITO SÉRIE:


### a. Declare as variáveis do transformador ($R_1$, $R_2$, $L_1$, $L_2$) e da fonte (frequência e tensão eficaz - Ufonte)

In [69]:
#Resistências 
R1 = 0.5
R2 = 0.5

#Indutâncias
L1 = 1e-6
L2 = 1e-6

#Capacitâncias
C1 = 1e-6
C2 = 1e-6

#Frequências 
f = 36000 #Gabriel amigo do Vini falou que o Carrarreto falou na sala deles (CHECAR)

#Tensões
V1 = 5*2/np.pi

#Frequência Angular:
w = 2*np.pi*f

ka = 0.2 #fator de acoplamento CHECAR
M = ka*((L1*L2)**(1/2))



#Reatâncias
XL1 = w*L1*1j
XL2 = w*L2*1j

XM = 1j*w*M

XC1 = 1/(1j*w*C1)
XC2 = 1/(1j*w*C2)




In [70]:
#Valores da capacitância
capacitancias = [1.5e-7, 1e-7, 4.7e-7, 1e-6, 4.7e-6]

#valores da frequência
frequencias = list(range(100, 100001, 100))

#Valores da frequencia de resonancia
frequencias_resonancias = []




In [71]:
def CalculaResistenciaBobina(Uf):
    Zdc=[R1+XL1, XL2+R2]
    return Zdc[0], Zdc[1]

In [72]:
R_bobinas = []
Rdc1, Rdc2 = CalculaResistenciaBobina(V1)

for f in frequencias:
    R_bobina1 = Rdc1 + (Rdc1 * f * 1e-5)
    R_bobina2 = Rdc2 + (Rdc2 * f * 1e-5)
    R_bobinas.append([R_bobina1, R_bobina2, f])

print(R_bobinas)

[[(0.5005+0.22642086572952358j), (0.5005+0.22642086572952358j), 100], [(0.501+0.22664706040058205j), (0.501+0.22664706040058205j), 200], [(0.5015+0.2268732550716405j), (0.5015+0.2268732550716405j), 300], [(0.502+0.22709944974269897j), (0.502+0.22709944974269897j), 400], [(0.5025+0.22732564441375744j), (0.5025+0.22732564441375744j), 500], [(0.503+0.2275518390848159j), (0.503+0.2275518390848159j), 600], [(0.5035+0.22777803375587435j), (0.5035+0.22777803375587435j), 700], [(0.504+0.22800422842693283j), (0.504+0.22800422842693283j), 800], [(0.5045+0.2282304230979913j), (0.5045+0.2282304230979913j), 900], [(0.505+0.22845661776904977j), (0.505+0.22845661776904977j), 1000], [(0.5055+0.2286828124401082j), (0.5055+0.2286828124401082j), 1100], [(0.506+0.22890900711116668j), (0.506+0.22890900711116668j), 1200], [(0.5065+0.22913520178222516j), (0.5065+0.22913520178222516j), 1300], [(0.507+0.22936139645328363j), (0.507+0.22936139645328363j), 1400], [(0.5075+0.22958759112434207j), (0.5075+0.22958759

In [73]:
def CalcularTransformadorSerie(Uf, Rc, Rbobina1, Rbobina2):
    Z=np.array([[R_bobina1+XC1, -XM],[-XM, R_bobina2+Rc+XC2]])
    V=np.array([Uf,0])
    i=np.dot(linalg.inv(Z),V)
    voltagem_2 = i[1].real *Rc
    return i[0], i[1], voltagem_2

In [74]:
def CalcularTransformadorParalelo(Uf, Rc, Rbobina1, Rbobina2):
    Xep = (Rc*XC2)/(Rc+XC2)
    Z=np.array([[R_bobina1+XC1, -XM],[-XM, R_bobina2+Xep]])
    V=np.array([Uf,0])
    i=np.dot(linalg.inv(Z),V)
    voltagem_2 = i[1].real*Rc
    return i[0], i[1], voltagem_2

In [75]:
#Calcula o valor da indutância dependendo da capacitância e da frequência angular:
def CalculaIndutância(C,w):
    L = 1/((w**2)*C)
    return L

#Calcula o valor da Frequência Angular dependendo da frequência
def CalculaFrequenciaAngular(f):
    w = 2*np.pi*f
    return w

In [76]:
RC = 5 # O que o prof faz no código do Lab --> NÂO TEMOS CERTEZA
frequencias_dic = {}
for resistencias in R_bobinas:
    R_bobina1 = resistencias[0]
    R_bobina2 = resistencias[1]
    f = resistencias[2]
    i1s, i2s, V2s = CalcularTransformadorSerie(V1,RC, R_bobina1, R_bobina2)
    i1p, i2p, V2p = CalcularTransformadorParalelo(V1,RC, R_bobina1, R_bobina2)
    frequencias_dic[f] = [i1s, i2s, V2s], [i1p, i2p, V2p]
    print(f'----------------------------------\nPARALELO (f: {f}Hz) \no valor de i1 é: {i1p.round(4)} \no valor de i2 é: {i2p.round(4)} \no valor de V2 é: {V2p.round(4)} \n')
    print(f'SERIE \no valor de i1 é: {i1s.round(4)} \no valor de i2 é: {i2s.round(4)} \no valor de V2 é: {V2s.round(4)} \n----------------------------------\n')

----------------------------------
PARALELO (f: 100Hz) 
o valor de i1 é: (0.0894+0.7483j) 
o valor de i2 é: (-0.0081-0.0053j) 
o valor de V2 é: -0.0406 

SERIE 
o valor de i1 é: (0.0893+0.7482j) 
o valor de i2 é: (-0.0042-0.0025j) 
o valor de V2 é: -0.0212 
----------------------------------

----------------------------------
PARALELO (f: 200Hz) 
o valor de i1 é: (0.0895+0.7483j) 
o valor de i2 é: (-0.0081-0.0053j) 
o valor de V2 é: -0.0406 

SERIE 
o valor de i1 é: (0.0894+0.7483j) 
o valor de i2 é: (-0.0042-0.0025j) 
o valor de V2 é: -0.0212 
----------------------------------

----------------------------------
PARALELO (f: 300Hz) 
o valor de i1 é: (0.0896+0.7483j) 
o valor de i2 é: (-0.0081-0.0053j) 
o valor de V2 é: -0.0406 

SERIE 
o valor de i1 é: (0.0895+0.7483j) 
o valor de i2 é: (-0.0042-0.0025j) 
o valor de V2 é: -0.0212 
----------------------------------

----------------------------------
PARALELO (f: 400Hz) 
o valor de i1 é: (0.0897+0.7483j) 
o valor de i2 é: (-0.0081-0

Loop para plotar os gráficos

In [77]:
#Eficiencias para transformador em série
eficiencias_serie = []

#Eficiencias para transformador em paralelo
eficiencias_paralelo = []

dic_serie = {}
dic_paralelo = {}

for C in capacitancias:
    dic_serie[C] = []
    dic_paralelo[C] = []

    for f, valores in frequencias_dic.items():
        i1s, i2s, V2s = valores[0]
        i1p, i2p, V2p = valores[1]

        w = CalculaFrequenciaAngular(f)
        L = CalculaIndutância(C,w)

        f_ressonacia = 1/(2*pi*((L*C)**.5))

        PsaidaS = 0.5*V2s.real*i2s.real
        PentradaS = 0.5*V1.real*i1s.real
        eficienciaS = PsaidaS/PentradaS
        eficiencias_serie.append(eficienciaS)
        
        dic_serie[C].append([f_ressonacia, eficienciaS, PsaidaS])
        
        PsaidaP = 0.5*V2p.real*i2p.real
        PentradaP = 0.5*V1.real*i1p.real
        eficienciaP = PsaidaP/PentradaP
        eficiencias_paralelo.append(eficienciaP)
        dic_paralelo[C].append([f_ressonacia, eficienciaP, PsaidaP])

print(dic_paralelo)


[[100.0, 0.001161000160916646, 0.0001651387911217798], [200.0, 0.0011600853402476436, 0.0001651862915463733], [300.0, 0.0011591721123848423, 0.00016523376105518847], [400.0, 0.0011582604725223435, 0.00016528119962897948], [500.0, 0.0011573504158735133, 0.00016532860724852253], [600.0, 0.0011564419376708858, 0.00016537598389461568], [699.9999999999999, 0.0011555350331660715, 0.00016542332954807922], [800.0, 0.0011546296976296577, 0.00016547064418975547], [900.0, 0.0011537259263511238, 0.00016551792780050936], [1000.0, 0.0011528237146387343, 0.0001655651803612273], [1100.0000000000002, 0.0011519230578194582, 0.00016561240185281831], [1200.0, 0.0011510239512388695, 0.00016565959225621344], [1299.9999999999998, 0.0011501263902610587, 0.0001657067515523659], [1399.9999999999998, 0.0011492303702685387, 0.00016575387972225087], [1500.0, 0.0011483358866621577, 0.0001658009767468661], [1600.0, 0.0011474429348610046, 0.00016584804260723124], [1699.9999999999998, 0.0011465515103023258, 0.00016589